In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from qutip import *

#### 1. Operators and System
In this script we consinder the joint space of:
- cavity supporting mode $a$ in which a single two-level atom is placed
- N dimensional space of the photon field 

Operators in the joint space are constructed using tensor products of corresponding operators in the separate spaces in the following function:

In [3]:
def operators(kappa, gamma, g, N, E):
    # input: various parameters 
    # output: a dictionary of joint operators for the system

    # Anhilation Operators
    a  = tensor(destroy(N), qeye(2))    # a - cavity lowering operator
    sm = tensor(qeye(N), sigmam())    # sigma - atomic lowering operator

    # Collapse Operators
    C1 = np.sqrt(2*kappa)*a
    C2 = np.sqrt(gamma)*sm
    C1dC1 = C1.dag() * C1
    C2dC2 = C2.dag() * C2
    return locals()

#### 2. Superoperators and density matrix equations
The generic form of a master equation is:
$$
\frac{d\rho}{dt} = \hat{\mathcal{L}}\rho
$$
where $\rho$ is the density matrix and Liouvillian $\mathcal{L}$ is a superoperator which may involve both premultiplication and postmultiplication by other operators. For our joint system, the Liouvillian is given by:
$$
\hat{\mathcal{L}} \rho = \gamma \hat{\sigma}_- \rho \hat{\sigma}_+
- \frac{\gamma}{2} \hat{\sigma}_- \hat{\sigma}_+ \rho 
- \frac{\gamma}{2} \rho \hat{\sigma}_+ \hat{\sigma}_-
$$

When considering the operation of $\hat{\mathcal{L}}$ on matrix $\rho$, we need to perform calculations 'flatly'. This means that we need to:
1. 'flatten' the matrix $\rho$ into a column vector $\tilde{\rho}$ 
2. convert the operators in $\hat{\mathcal{L}}$ into superoperators of the corresponding dimensions to $\tilde{\rho}$. 

This is done by using the spre() and spost() functions in QuTiP which correspondingly gives the superoperaotrs corresponding to premultiplication and postmultiplication of the input operator.If we need to consider the Hamiltonian in the evolution as well, we can also add the commutator to the Liouvillian as such:
$$
\hat{\mathcal{L}}_{\hat{H}\rho} = -i[\hat{H},\rho] = -i(\hat{H}\rho-\rho\hat{H} )
$$
These steps are performed in the following function:

In [4]:
def Liouv( Del_c, a, sm, C1, C2, C1dC1, C2dC2 ):
    # Hamiltonian
    H = Del_c * sm.dag() * sm + Del_c * a.dag() * a + 1j*g*(a.dag()*sm - sm.dag()*a) + E*(a.dag()+a)

    # Liovillian
    LH = -1j * (spre(H) - spost(H))
    L1 = spre(C1)*spost(C1.dag()) - 0.5*spre(C1dC1) - 0.5*spost(C1dC1)
    L2 = spre(C2)*spost(C2.dag()) - 0.5*spre(C2dC2) - 0.5*spost(C2dC2)
    L = LH+L1+L2

    return L 

#### 3. Calculating Operator expectation values
The expectation value of some operator $\hat{A}$ in the system can be obtained from:
- demsity matrix $\rho$: &ensp; $\langle \hat{A} \rangle = Tr(\hat{A}\rho)$. 
- state ket $\ket{\phi}$: &emsp;&emsp;&ensp; $\langle \hat{A} \rangle = Tr(\hat{A}\ket{\phi}\bra{\phi}) = \bra{\phi}\hat{A}\ket{\phi} $

This can be performed by the QuTiP function expect(op,state). In this specific example, we can obtain the steady state solution of $\rho$ in the master equation using the steady() function.